OPX sequence for release and recapture with additional D1 light. Fixed release time, no collision.

In [217]:
from qm.qua import *
from qm import QuantumMachinesManager, SimulationConfig
from quam.components import BasicQuAM, SingleChannel
from quam.components.pulses import SquarePulse
from qualang_tools.units import unit
from QuAM_utilities import *

import matplotlib.pyplot as plt
import time

u = unit() 

In [218]:
machine = BasicQuAM()

# AOM RFs
cool_double_rf = 200*u.MHz
repump_double_rf = 200*u.MHz

# TWEEZER LOADING
tweez_central_freq = 75.1*u.MHz
tweez_difference_freq = 15*u.MHz
tweez_low_freq = tweez_central_freq - tweez_difference_freq/2
tweez_high_freq = tweez_central_freq + tweez_difference_freq/2

tweez_ramp_amplitude = 0.04
tweez_ramp_time = 100*u.ms #
tweez_plateau_amplitude = tweez_ramp_amplitude
tweez_plateau_time = 150.15*u.ms #

# D1 EIT
D1_EIT_time = 15*u.ms
tweez_square_amplitude_EIT = 0.04

# GAP 1
gap1_time = 5*u.ms #200*u.ns#

# COLLISION
coll_time = 80*u.ms #100*u.us #
tweez_square_amplitude_coll = 0.15
repump_square_amplitude_coll_double = 0.38 # fixed
cool_square_amplitude_coll_double = 0.38 # fixed
coll_modulation_frequency = 1.7*u.MHz
tweez_duty_cycle = 0.4 # high fraction for tweezer
rc_duty_cycle = 0.3 # high fraction for repumper and cooler
rc_delay_fraction = 0.05 # delay for repumper and cooler (fraction of period) 
                        # the sum of the repumper/cooler duty cycles must be less than 1

# GAP 2
t_release = 0.1*u.us
gap2_time = 5*u.ms

# IMAGING
# the duty cycle parameters are the same as for collision
imag_time = 80*u.ms #
tweez_square_amplitude_imag = 0.15
repump_square_amplitude_imag_double = 0.38 # fixed
cool_square_amplitude_imag_double = 0.38 # fixed

# DELAY FOR IMAGING
delay_imaging_time = 250.15*u.ms #

# FOR CONTINUOUSLY DRIVEN AOMs: cooler/repumper single pass & two spots tweezers
# this is for amplitude modulation
continuous_time = 50*u.ms
continuous_time_long_pulse = 1*u.s

# collision
repump_square_amplitude_coll_single = 0.065 #0.05
cool_square_amplitude_coll_single = 0.12 #0.07

# imaging
repump_square_amplitude_imag_single = 0.06         
cool_square_amplitude_imag_single = 0.06
spot_amplitude1 = 0.38 # fixed
spot_amplitude2 = 0.38 # fixed

In [221]:
# additional dependent variables useful for pulse definition
# NB: the time durations must be multiples of the clock cycle!

# these are not used in the function, but are printed for debug
tweez_ramp_step_time = 40*u.ns
n_steps = round(tweez_ramp_time/tweez_ramp_step_time)
plateau_intermediate_duration = 10*u.us
plateau_num_portions = round(tweez_plateau_time/plateau_intermediate_duration)

# these are in seconds, multiply by u.s before passing as arguments
coll_modulation_period = 1/coll_modulation_frequency
tweez_high_time = 1/coll_modulation_frequency*tweez_duty_cycle
tweez_low_time = 1/coll_modulation_frequency*(1-tweez_duty_cycle)
rc_high_time = 1/coll_modulation_frequency*rc_duty_cycle
rc_delay_time = 1/coll_modulation_frequency*rc_delay_fraction
rc_low_time = coll_modulation_period-rc_delay_time-rc_high_time
number_of_periods_collision = round(coll_time/(coll_modulation_period*u.s)) # used as argument

imag_modulation_period = coll_modulation_period

number_of_periods_imaging = round(imag_time/(imag_modulation_period*u.s)) # used as argument

# for debugging
print(f"\n Number of steps for the tweezer ramp: {n_steps}, each of {tweez_ramp_step_time} ns")
print(f"Number of plateau portions: {plateau_num_portions}")
print("Number of collision periods:", number_of_periods_collision)
print("Number of imaging periods:", number_of_periods_imaging, "\n")
print(f"Low duration for tweezer: {round(tweez_low_time*u.s/4)*4}ns")
print(f"High duration for tweezer: {round(tweez_high_time*u.s/4)*4}ns")
print(f"Total: {round(tweez_low_time*1e9/4)*4 + round(tweez_high_time*u.s/4)*4}ns \n")
print(f"Delay duration for repumper/cooler: {round(rc_delay_time*u.s/4)*4}ns")
print(f"High duration for repumper/cooler: {round(rc_high_time*u.s/4)*4}ns")
print(f"Low duration for repumper/cooler: {round(rc_low_time*u.s/4)*4}ns")
print(f"Total: {round(rc_delay_time*u.s/4)*4 + round(rc_low_time*u.s/4)*4 + round(rc_high_time*u.s/4)*4}ns \n")

# add a warning if the durations are not the same


 Number of steps for the tweezer ramp: 2500000, each of 40.0 ns
Number of plateau portions: 15015
Number of collision periods: 136000
Number of imaging periods: 136000 

Low duration for tweezer: 352ns
High duration for tweezer: 236ns
Total: 588ns 

Delay duration for repumper/cooler: 28ns
High duration for repumper/cooler: 176ns
Low duration for repumper/cooler: 384ns
Total: 588ns 



In [222]:
machine.channels['AOM_single_repumper'] = AOM_single_repumper = SingleChannel(opx_output=('con1', 1), intermediate_frequency=200e6)
machine.channels['AOM_double_repumper'] = AOM_double_repumper = SingleChannel(opx_output=('con1', 2), intermediate_frequency=repump_double_rf)
machine.channels['AOM_single_cooler'] = AOM_single_cooler = SingleChannel(opx_output=('con1', 3), intermediate_frequency=200e6)
machine.channels['AOM_double_cooler'] = AOM_double_cooler = SingleChannel(opx_output=('con1', 4), intermediate_frequency=cool_double_rf)
machine.channels['AOM_tweez_mod'] = AOM_tweez_mod = SingleChannel(opx_output=('con1', 5), intermediate_frequency=89.161e6) # for tweezer modulation
machine.channels['AOM_tweez_spots1'] = AOM_tweez_spots1 = SingleChannel(opx_output=('con1', 6), intermediate_frequency=tweez_high_freq) # first component for generating two spots
machine.channels['AOM_tweez_spots2'] = AOM_tweez_spots2 = SingleChannel(opx_output=('con1', 6), intermediate_frequency=tweez_low_freq) # second component for generating two spots

AOM_single_repumper.operations['repump_continuous_coll'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=repump_square_amplitude_coll_single)
AOM_single_repumper.operations['repump_continuous_imag'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=repump_square_amplitude_imag_single)
AOM_single_cooler.operations['cool_continuous_coll'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=cool_square_amplitude_coll_single)
AOM_single_cooler.operations['cool_continuous_imag'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=cool_square_amplitude_imag_single)

AOM_double_repumper.operations['repump_high_pulse_coll'] = SquarePulse(length=round(rc_high_time*u.s/4)*4, amplitude=repump_square_amplitude_coll_double)
AOM_double_repumper.operations['repump_high_pulse_imag'] = SquarePulse(length=round(rc_high_time*u.s/4)*4, amplitude=repump_square_amplitude_imag_double)
AOM_double_repumper.operations['repump_low_pulse'] = SquarePulse(length=round(rc_low_time*u.s/4)*4, amplitude=0)
AOM_double_cooler.operations['cool_high_pulse_coll'] = SquarePulse(length=round(rc_high_time*u.s/4)*4, amplitude=cool_square_amplitude_coll_double)
AOM_double_cooler.operations['cool_high_pulse_imag'] = SquarePulse(length=round(rc_high_time*u.s/4)*4, amplitude=cool_square_amplitude_imag_double)
AOM_double_cooler.operations['cool_low_pulse'] = cool_low_pulse = SquarePulse(length=round(rc_low_time*u.s/4)*4, amplitude=0)

AOM_tweez_mod.operations['tweez_step_pulse'] = tweez_step_pulse = SquarePulse(length=round(tweez_ramp_step_time/4)*4, amplitude=tweez_ramp_amplitude) 
AOM_tweez_mod.operations['tweez_plateau_pulse'] = SquarePulse(length=100, amplitude=tweez_plateau_amplitude) # this value is overridden by the time/slice value
AOM_tweez_mod.operations['tweez_high_pulse_coll'] = SquarePulse(length=round(tweez_high_time*u.s/4)*4, amplitude=tweez_square_amplitude_coll)
AOM_tweez_mod.operations['tweez_high_pulse_imag'] = SquarePulse(length=round(tweez_high_time*u.s/4)*4, amplitude=tweez_square_amplitude_imag)
AOM_tweez_mod.operations['tweez_high_pulse_EIT'] = SquarePulse(length=round(D1_EIT_time/4)*4, amplitude=tweez_square_amplitude_EIT)
AOM_tweez_mod.operations['tweez_low_pulse'] = SquarePulse(length=round(tweez_low_time*u.s/4)*4, amplitude=0)

#AOM_tweez_spots1.operations['tweez_continuous'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=spot_amplitude1)
#AOM_tweez_spots2.operations['tweez_continuous'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=spot_amplitude2)
AOM_tweez_spots1.operations['tweez_continuous'] = SquarePulse(length=100, amplitude=spot_amplitude1)
AOM_tweez_spots2.operations['tweez_continuous'] = SquarePulse(length=100, amplitude=spot_amplitude2)


In [223]:
qua_config = machine.generate_config()

Write the QUA program

In [225]:
with program() as ensemble_release:

    with infinite_loop_():
        AOM_tweez_spots1.play("tweez_continuous") 
        AOM_tweez_spots2.play("tweez_continuous")
    
    with infinite_loop_():
        wait_for_trigger('AOM_tweez_mod') 
        wait_for_trigger('AOM_double_repumper')
        wait_for_trigger('AOM_double_cooler')
        wait_for_trigger('AOM_single_repumper')
        wait_for_trigger('AOM_single_cooler')

        # atoms loading
        long_ramp('tweez_step_pulse', 'AOM_tweez_mod', tweez_ramp_time)
        long_pulse('tweez_plateau_pulse', 'AOM_tweez_mod', tweez_plateau_time)
        align('AOM_tweez_mod', 'AOM_double_cooler', 'AOM_double_repumper','AOM_single_cooler','AOM_single_repumper')
        
        # gap1
        play('tweez_step_pulse', 'AOM_tweez_mod', duration=round(gap1_time/4))
        play('cool_low_pulse', 'AOM_double_cooler', duration=round(gap1_time/4))
        play('repump_low_pulse', 'AOM_double_repumper', duration=round(gap1_time/4))
    
        long_pulse('cool_continuous_coll', 'AOM_single_cooler', D1_EIT_time+gap1_time)
        long_pulse('repump_continuous_coll', 'AOM_single_repumper', D1_EIT_time+gap1_time)

        # D1 light for cooling
        play('tweez_high_pulse_EIT', 'AOM_tweez_mod', duration=round(D1_EIT_time/4))
        play('cool_low_pulse', 'AOM_double_cooler', duration=round(D1_EIT_time/4))
        play('repump_low_pulse', 'AOM_double_repumper', duration=round(D1_EIT_time/4))
        #play('cool_high_pulse_coll', 'AOM_double_cooler', duration=round(D1_EIT_time/4))
        #play('repump_high_pulse_coll', 'AOM_double_repumper', duration=round(D1_EIT_time/4))

        #gap2
        play('tweez_step_pulse', 'AOM_tweez_mod', duration=round((gap2_time-t_release)/4))
        play('tweez_low_pulse', 'AOM_tweez_mod', duration=round(t_release/4))
        play('cool_low_pulse', 'AOM_double_cooler', duration=round((gap2_time)/4))
        play('repump_low_pulse', 'AOM_double_repumper', duration=round((gap2_time)/4))

        long_pulse('cool_continuous_imag', 'AOM_single_cooler', gap2_time+imag_time)
        long_pulse('repump_continuous_imag', 'AOM_single_repumper', gap2_time+imag_time)

        # imaging
        stroboscopic(('tweez_low_pulse','tweez_high_pulse_imag'), 'AOM_tweez_mod', number_of_periods_imaging)
        stroboscopic(('repump_high_pulse_imag','repump_low_pulse'), 'AOM_double_repumper', number_of_periods_imaging, rc_delay_time)
        stroboscopic(('cool_high_pulse_imag','cool_low_pulse'), 'AOM_double_cooler', number_of_periods_imaging, rc_delay_time)

        # delay for image
        long_pulse('tweez_high_pulse_imag', 'AOM_tweez_mod', delay_imaging_time)
        long_pulse('cool_low_pulse', 'AOM_double_cooler', delay_imaging_time)
        long_pulse('repump_low_pulse', 'AOM_double_repumper', delay_imaging_time)

        long_pulse('cool_continuous_imag', 'AOM_single_cooler', delay_imaging_time + imag_time)
        long_pulse('repump_continuous_imag', 'AOM_single_repumper', delay_imaging_time + imag_time)
                
        # imaging
        stroboscopic(('tweez_low_pulse','tweez_high_pulse_imag'), 'AOM_tweez_mod', number_of_periods_imaging)
        stroboscopic(('cool_high_pulse_imag','cool_low_pulse'), 'AOM_double_cooler', number_of_periods_imaging, rc_delay_time)
        stroboscopic(('repump_high_pulse_imag','repump_low_pulse'), 'AOM_double_repumper', number_of_periods_imaging, rc_delay_time)


Open the machine, load the state and run the program

In [227]:
ip_address = '130.79.148.167'
qmm = QuantumMachinesManager(host=ip_address, log_level='DEBUG')


2024-12-16 17:39:36,474 - qm - DEBUG    - Probing gateway at: 130.79.148.167:80
2024-12-16 17:39:37,528 - qm - DEBUG    - Connection redirected from '130.79.148.167:80' to '130.79.148.167:10010'
2024-12-16 17:39:37,586 - qm - DEBUG    - Established connection to 130.79.148.167:10010
2024-12-16 17:39:37,587 - qm - DEBUG    - Gateway discovered at: 130.79.148.167:80
2024-12-16 17:39:37,723 - qm - INFO     - Performing health check
2024-12-16 17:39:37,736 - qm - INFO     - Health check passed


In [228]:
%matplotlib qt

Simulation = False

if Simulation == True:

    start_time = time.time()
    simulated_job = qmm.simulate(qua_config, ensemble_release, SimulationConfig(duration=int(20000/4)))
    end_time = time.time()
    print(f"Simulation time: {end_time - start_time} s")

    # get DAC and digital samples
    samples = simulated_job.get_simulated_samples()
    plt.figure()
    #plt.plot(samples.con1.analog["1"], "--", label="sg_repumper")
    #plt.plot(samples.con1.analog["2"], "-", label="db_repumper")
    #plt.plot(samples.con1.analog["3"], "--", label="sg_cooler")
    #plt.plot(samples.con1.analog["4"], "-", label="db_cooler")
    plt.plot(samples.con1.analog["5"], "-", label="tweezer_mod")
    #plt.plot(samples.con1.analog["6"], "-", label="tweezer_beat")
    plt.legend()
    plt.xlabel("Time [ns]")
    plt.ylabel("Signal [V]") 
    plt.show()
else:
    qm = qmm.open_qm(qua_config)
    job = qm.execute(ensemble_release)

2024-12-16 17:39:39,229 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing Quantum Machines: 
2024-12-16 17:39:39,230 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1734367077538 is canceling running job 1730223365056.
2024-12-16 17:39:39,239 - qm - INFO     - Sending program to QOP for compilation
2024-12-16 17:39:40,321 - qm - INFO     - Executing program


In [38]:
simulated_job.execution_report()

Execution report for job 1730223364894
No errors

In [229]:
job.halt()

True